<a href="https://colab.research.google.com/github/NSCC-HydroPandas/Canadian-Tidal-Data-Metadat/blob/main/Creating_Benchmark_Reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Station Benchmark Reports

**1. Download and Install VS Code and Python 3.11.9**
- [VS Code](https://apps.microsoft.com/detail/xp9khm4bk9fz7q?hl=en-us&gl=US)
- [Python](https://www.python.org/ftp/python/3.11.9/python-3.11.9-embed-amd64.zip)

**2. Download jupyter Notebooks**
- Select the notebook (.pynb) file to download and select the download icon

**3. Open Notebook with VS code**
- If default program double click file
- Open from VS Code (Open Folder or Open File)

**4. Run the Notebook**
- If first Time running please run the pip install cell
- One cell at a time or Run All
- Inputs
- Provide inputs according to the Notebook Type

# 0. Set up Enviroment

In [ ]:
!pip install requests
!pip install pandas
import pandas as pd 
import requests
import json
from os import mkdir, path

# 1. User Inputs

In [ ]:
def user_inputs():
    stn = int(input('Provide the Station Number '))
    stn = f"{stn:05d}"
    user_dir = input('Provide the directory path ')
    if path.exists(user_dir + '/' + stn):
        pass
    else:
        mkdir(user_dir + '/' + stn)
    if path.exists(user_dir + '/' + stn + '/BenchMarks_Reports'):
        pass
    else:
        mkdir(user_dir + '/' + stn + '/BenchMarks_Reports')
    if path.exists(user_dir + '/' + stn + '/BenchMarks_Reports/Images'):
        pass
    else:
        mkdir(user_dir + '/' + stn + '/BenchMarks_Reports/Images')
    if path.exists(user_dir + '/' + stn + '/Station_Report'):
        pass
    else:
        mkdir(user_dir + '/' + stn + '/Station_Report')
    if path.exists(user_dir + '/' + stn + '/Station_Report/Images'):
        pass
    else:
        mkdir(user_dir + '/' + stn + '/Station_Report/Images')
    return [stn, user_dir]

# 2. Station DataFrame

In [ ]:
user_input = user_inputs()
stn = user_input[0]
user_dir = user_input[1]


# 3. Get Station Endpoint from Station Number

In [ ]:
station_summary_url = f'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations?code={stn}'

In [ ]:
df = pd.read_json(station_summary_url)
stid = df['id'].values[0]
stid

# 4. Get Benchmarks for Station

In [ ]:
bm_url = (f'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/benchmarks?stationId={stid}')
bms = pd.read_json(bm_url)
bms

# 5. Get Benchmark Metadata

In [ ]:
bms_metadata = pd.DataFrame()

for bmid in bms.id:
    bm_metadata_url = f'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/benchmarks/{bmid}/metadata'
    r = requests.get(bm_metadata_url)
    json_str = r.content.decode('utf-8')

    md = json.loads(json_str)

    temp_meta = pd.DataFrame([md])

    bms_metadata = pd.concat([bms_metadata, temp_meta], ignore_index=True)

bms_metadata

bms_metadata = bms_metadata.fillna("NULL")

# 6. Get Elevation Data

In [ ]:
bms_elevations = pd.DataFrame()
for bmid in bms_metadata.id:
    elev_url = f'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/benchmarks/{bmid}/elevations'
    r = requests.get(elev_url)
    json_str = r.content.decode('utf-8')
    el = json.loads(json_str)
    temp_elev = pd.DataFrame(el)
    bms_elevations = pd.concat([bms_elevations, temp_elev], ignore_index=True)

bms_elevations = bms_elevations.fillna("NULL")

# 8. Create BM HTML Pages

In [ ]:
for idx, row in bms_metadata.iterrows():
    html_content = f"""
    <html>
    <head>
        <title>Station {stn} Metadata - Benchmark {row.get('benchmarkNumber', row.get('id', idx))} Metadata</title>
    </head>
    <body>
        <h1>Station {stn} : {df['officialName'].values[0]}</br>
        Benchmark {row.get('benchmarkNumber', row.get('id', idx))}</h1>
        <ul>
    """
    # Add all metadata fields dynamically
    for col in bms_metadata.columns:
        html_content += f"<li><strong>{col}:</strong> {row[col]}</li>"

    # Add elevation info for this benchmark
    elev_rows = bms_elevations[bms_elevations['id'] == row['id']]
    for _, elev_row in elev_rows.iterrows():
        html_content += "<li><strong>Elevation Record:</strong></li><ul>"
        for col in elev_row.index:
            html_content += f"<li>{col}: {elev_row[col]}</li>"
        html_content += "</ul>"

    html_content += "</ul>"
    
    html_content += """
    </body>
    </html>
    """
    filename = f"{row.get('benchmarkNumber', row.get('id', idx))}_metadata.html"
    with open(f'{user_dir}\\{stn}\\BenchMarks_Reports\\{filename}', "w", encoding="utf-8") as f:
        f.write(html_content)
